In [1]:
import logging
from utility import add_project_root_to_path

logging.basicConfig(level=logging.ERROR)

add_project_root_to_path()

In [2]:
# Building experiments configs

from experiments.configs import DATA_SOURCE_BY_ALIAS, DEFAULT_UNINFORMED_USERS_CONFIG
from experiments.experiment import Experiment
from user.informed_user import InformedUser

from fee_algorithm.fixed_fee import FixedFee
from fee_algorithm.discrete_fee_perfect_oracle import DiscreteFeePerfectOracle
from fee_algorithm.based_on_trade_count_fee import BasedOnTradeCountFee
from fee_algorithm.adaptive_fee_based_on_block_price_move import AdaptiveBasedOnPreviousBlockPriceMoveFee
from experiments.run_multiple_experiments import run_multiple_experiments, get_experiment_key
from fee_algorithm.dynamic_fee_amm_for_amm import AMMforAMMfee


from copy import deepcopy

fee_algos_to_consider = {
    "fixed_fee": FixedFee(exchange_fee_rate=0.003), # 30 bps
    "based_on_trade_count_fee": BasedOnTradeCountFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "adaptive_based_on_previous_block_price_move_fee": AdaptiveBasedOnPreviousBlockPriceMoveFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "discrete_fee_perfect_oracle": DiscreteFeePerfectOracle(fee_rate_in_arbitrage_direction=0.0045, fee_rate_in_non_arbitrage_direction=0.0015), # 45/15 bps
    "AMM_fee": AMMforAMMfee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003),
}

/Users/ir.v.lebedeva/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import pandas as pd

df_stable_not_stable = pd.read_csv('df_stable_not_stable.csv')
df_stable_stable = pd.read_csv('df_stable_stable.csv')
df_not_stable_not_stable = pd.read_csv('df_not_stable_not_stable.csv')

In [4]:
df_not_stable_not_stable.head()

,Unnamed: 0,Period,Pair,Slope,Std_dev,start_time,end_time,Market_type,A_symbol,B_symbol
0,0,2024-05-06/2024-05-12,ETH/BTC,-0.000134,0.002148,2024-05-06,2024-05-12 23:59:59.999999999,calm_market,ETH,BTC
1,1,2024-05-13/2024-05-19,ETH/BTC,-0.000133,0.002753,2024-05-13,2024-05-19 23:59:59.999999999,calm_market,ETH,BTC
2,2,2024-05-20/2024-05-26,ETH/BTC,0.000825,0.008224,2024-05-20,2024-05-26 23:59:59.999999999,bull_market,ETH,BTC
3,3,2024-05-27/2024-06-02,ETH/BTC,-0.000088,0.002613,2024-05-27,2024-06-02 23:59:59.999999999,calm_market,ETH,BTC
4,4,2024-06-03/2024-06-09,ETH/BTC,-0.000237,0.002042,2024-06-03,2024-06-09 23:59:59.999999999,calm_market,ETH,BTC


In [5]:
df_not_stable_not_stable['start_time'] = pd.to_datetime(df_not_stable_not_stable['start_time'])
df_not_stable_not_stable['end_time'] = pd.to_datetime(df_not_stable_not_stable['end_time'])

df_stable_not_stable['start_time'] = pd.to_datetime(df_stable_not_stable['start_time'])
df_stable_not_stable['end_time'] = pd.to_datetime(df_stable_not_stable['end_time'])

df_stable_stable['start_time'] = pd.to_datetime(df_stable_stable['start_time'])
df_stable_stable['end_time'] = pd.to_datetime(df_stable_stable['end_time'])

### Stable/Non-Stable Pairs

In [6]:
from experiments.experiment import HistoricalDataDescription

experiment_configs = {}
data = df_stable_not_stable.copy()

for i, row in data.iterrows():
    data_source = HistoricalDataDescription(start_time=row['start_time'],
                                            end_time=row['end_time'], 
                                            A_symbol=row['A_symbol'], 
                                            B_symbol=row['B_symbol'])
    for fee_algo_alias, fee_algo in fee_algos_to_consider.items():
        experiment_configs[
            get_experiment_key(
                {
                    "period_alias": row['Market_type'],
                    "fee_algorithm": fee_algo_alias,
                }
            )
        ] = Experiment(
            data=data_source,
            fee_algorithm=deepcopy(fee_algo),
            uninformed_users=deepcopy(DEFAULT_UNINFORMED_USERS_CONFIG),
            informed_user=InformedUser(),
        )

In [7]:
from pprint import pprint

pprint(experiment_configs[
    get_experiment_key(
        {
        "period_alias": "volatile_market",
        "fee_algorithm": "adaptive_based_on_previous_block_price_move_fee",
    })
])

Experiment(data=HistoricalDataDescription(start_time=Timestamp('2025-05-05 00:00:00'), end_time=Timestamp('2025-05-11 23:59:59.999999999'), A_symbol='ADA', B_symbol='USDC', stable_coin_symbol='USDT', candle_interval='1m'), fee_algorithm=AdaptiveBasedOnPreviousBlockPriceMoveFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003, fee_step=0.0001, prev_block_begin_a_to_b_price=None, prev_block_end_a_to_b_price=None, prev_block_begin_b_to_a_price=None, prev_block_end_b_to_a_price=None), informed_user=InformedUser(), uninformed_users=UninformedUsersConfig(uninformed_user=UninformedUser(mu=0.00016, sigma=1e-05), probability_of_trade=0.5, n_users=1), initial_pool_value=25000000, network_fee=5, random_seed=0)


In [8]:
print(
    f"Total experiments to run: {len(experiment_configs)}"
)

Total experiments to run: 20


In [9]:
experiment_results_stable_not_stable = run_multiple_experiments(
    experiments=experiment_configs,
    parallel=True,
    max_workers=8,
)

Running Experiments:   0%|          | 0/20 [00:00<?, ?it/s]/Users/ir.v.lebedeva/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ir.v.lebedeva/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ir.v.lebedeva/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ir.v.lebedeva/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWa

A symbol candles count: 10081
B symbol candles count: 10081
Joined candles count: 10081
A symbol candles count: 10081
B symbol candles count: 10081
Joined candles count: 10081
A symbol candles count: 10081
B symbol candles count: 10081
Joined candles count: 10081


Running Experiments:  55%|█████▌    | 11/20 [00:17<00:21,  2.35s/it]

A symbol candles count: 10081
B symbol candles count: 10081
Joined candles count: 10081


Running Experiments:  60%|██████    | 12/20 [00:19<00:19,  2.39s/it]

...

Running Experiments:  75%|███████▌  | 15/20 [00:21<00:06,  1.27s/it]

A symbol candles count: 10081
B symbol candles count: 10081
Joined candles count: 10081
A symbol candles count: 10081
B symbol candles count: 10081
Joined candles count: 10081


Running Experiments:  90%|█████████ | 18/20 [00:26<00:02,  1.34s/it]

A symbol candles count: 10081
B symbol candles count: 10081
Joined candles count: 10081
A symbol candles count: 10081
B symbol candles count: 10081
Joined candles count: 10081


Running Experiments: 100%|██████████| 20/20 [00:35<00:00,  1.79s/it]


In [10]:
import pandas as pd
from visualizations.compare_fee_algoritms import get_experiments_summary_by_description

combined_df = get_experiments_summary_by_description(
    experiment_results_stable_not_stable,
).sort_values(by=["period_alias", "fee_algorithm"])

combined_df = combined_df.set_index(['period_alias'])

display(combined_df)

,fee_algorithm,iu_markout,iu_trade_count,iu_yield,uu_markout,uu_trade_count,uu_yield,lp_markout,lp_yield,impermanent_loss
period_alias,,,,,,,,,,
bear_market,AMM_fee,32513.00,1620,13.16,-56569.94,5085,-56.09,-9468.06,-2.72,21885.21
bear_market,adaptive_based_on_previous_block_price_move_fee,14035.72,793,12.35,-57019.36,5081,-56.61,13613.64,6.35,-3191.16
bear_market,based_on_trade_count_fee,14284.90,793,12.50,-57438.27,5081,-57.03,13783.37,6.41,-2146.76
bear_market,discrete_fee_perfect_oracle,11840.72,577,13.78,-57056.70,5057,-56.96,17045.99,9.16,-5588.95
bear_market,fixed_fee,14334.20,795,12.52,-57472.08,5079,-57.08,13767.88,6.40,-1443.99
bull_market,AMM_fee,29434.11,1612,11.91,-59285.76,5034,-56.24,-3378.35,-0.96,14000.78
bull_market,adaptive_based_on_previous_block_price_move_fee,9062.79,703,9.31,-58252.48,5063,-54.97,20359.69,10.01,-9814.94
bull_market,based_on_trade_count_fee,8700.50,699,9.06,-58665.93,5049,-55.53,21225.42,10.52,-8095.10
bull_market,discrete_fee_perfect_oracle,7014.50,509,9.69,-58951.25,5052,-55.71,24131.75,13.54,-12249.02


### Non-Stable/Non-Stable Pairs

In [16]:
from experiments.experiment import HistoricalDataDescription

experiment_configs = {}
data = df_not_stable_not_stable.copy()

for i, row in data.iterrows():
    data_source = HistoricalDataDescription(start_time=row['start_time'],
                                            end_time=row['end_time'], 
                                            A_symbol=row['A_symbol'], 
                                            B_symbol=row['B_symbol'])
    for fee_algo_alias, fee_algo in fee_algos_to_consider.items():
        experiment_configs[
            get_experiment_key(
                {
                    "period_alias": row['Market_type'],
                    "fee_algorithm": fee_algo_alias,
                }
            )
        ] = Experiment(
            data=data_source,
            fee_algorithm=deepcopy(fee_algo),
            uninformed_users=deepcopy(DEFAULT_UNINFORMED_USERS_CONFIG),
            informed_user=InformedUser(),
        )

In [ ]:
experiment_results_not_stable_not_stable = run_multiple_experiments(
    experiments=experiment_configs,
    parallel=True,
    max_workers=8,
)

In [13]:
import pandas as pd
from visualizations.compare_fee_algoritms import get_experiments_summary_by_description

combined_df = get_experiments_summary_by_description(
    experiment_results_not_stable_not_stable,
).sort_values(by=["period_alias", "fee_algorithm"])

combined_df = combined_df.set_index(['period_alias'])

display(combined_df)

,fee_algorithm,iu_markout,iu_trade_count,iu_yield,uu_markout,uu_trade_count,uu_yield,lp_markout,lp_yield,impermanent_loss
period_alias,,,,,,,,,,
bear_market,AMM_fee,7027.89,926,6.48,-57403.95,5088,-56.85,20306.06,9.70,-2270.49
bear_market,adaptive_based_on_previous_block_price_move_fee,1949.93,298,5.78,-56549.32,5022,-56.71,27999.39,20.98,-10601.16
bear_market,based_on_trade_count_fee,1771.05,278,5.64,-57294.25,5062,-57.02,28823.20,21.85,-10725.73
bear_market,discrete_fee_perfect_oracle,1262.37,194,5.70,-57051.30,5050,-56.93,29568.92,24.17,-10732.51
bear_market,fixed_fee,1967.85,286,5.97,-57033.71,5042,-56.96,28425.86,21.36,-10446.60
bull_market,AMM_fee,22196.32,1431,10.72,-58232.56,5033,-55.02,3716.24,1.19,83357.28
bull_market,adaptive_based_on_previous_block_price_move_fee,6876.92,597,8.59,-59240.74,5086,-55.42,23948.81,12.81,63904.21
bull_market,based_on_trade_count_fee,6927.49,618,8.47,-58686.91,5044,-55.36,23449.41,12.48,66062.74
bull_market,discrete_fee_perfect_oracle,5414.11,450,8.87,-58512.83,5046,-55.20,25618.71,15.34,61821.81


### Stable/Stable Pair

In [6]:
df_stable_stable.Pair.unique()

array(['TUSD/USDT', 'USDC/USDT', 'USDP/USDT'], dtype=object)

In [ ]:
df_stable_stable = df_stable_stable[~(df_stable_stable.Pair==['USDT/USDT'])]

In [6]:
from experiments.experiment import HistoricalDataDescription

experiment_configs = {}
data = df_stable_stable.copy()

for i, row in data.iterrows():
    data_source = HistoricalDataDescription(start_time=row['start_time'],
                                            end_time=row['end_time'], 
                                            A_symbol=row['A_symbol'], 
                                            B_symbol=row['B_symbol'])
    for fee_algo_alias, fee_algo in fee_algos_to_consider.items():
        experiment_configs[
            get_experiment_key(
                {
                    "period_alias": row['Market_type'],
                    "fee_algorithm": fee_algo_alias,
                }
            )
        ] = Experiment(
            data=data_source,
            fee_algorithm=deepcopy(fee_algo),
            uninformed_users=deepcopy(DEFAULT_UNINFORMED_USERS_CONFIG),
            informed_user=InformedUser(),
        )

In [7]:
experiment_results_stable_stable = run_multiple_experiments(
    experiments=experiment_configs,
    parallel=True,
    max_workers=8,
)

Running Experiments:   0%|          | 0/5 [00:00<?, ?it/s]/Users/ir.v.lebedeva/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ir.v.lebedeva/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ir.v.lebedeva/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ir.v.lebedeva/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWar

USDPUSDT
USDPUSDT
USDPUSDT
USDPUSDT
USDPUSDT


/Users/ir.v.lebedeva/Documents/Научка/DEX_new/Dex-Dynamic-Fee/visualizations/../historical_data.py:43: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  time_range = pd.date_range(start=start_time, end=end_time, freq=interval)
Running Experiments:   0%|          | 0/5 [00:04<?, ?it/s]

Symbol USDTUSDT is USDT/USDT - returning price=1
A symbol candles count: 8269
B symbol candles count: 0
Joined candles count: 0
Symbol USDTUSDT is USDT/USDT - returning price=1
A symbol candles count: 8269
B symbol candles count: 0
Joined candles count: 0
Symbol USDTUSDT is USDT/USDT - returning price=1
A symbol candles count: 8269
B symbol candles count: 0
Joined candles count: 0
Symbol USDTUSDT is USDT/USDT - returning price=1
A symbol candles count: 8269
B symbol candles count: 0
Joined candles count: 0



/Users/ir.v.lebedeva/Documents/Научка/DEX_new/Dex-Dynamic-Fee/visualizations/../historical_data.py:43: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  time_range = pd.date_range(start=start_time, end=end_time, freq=interval)
/Users/ir.v.lebedeva/Documents/Научка/DEX_new/Dex-Dynamic-Fee/visualizations/../historical_data.py:43: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  time_range = pd.date_range(start=start_time, end=end_time, freq=interval)
/Users/ir.v.lebedeva/Documents/Научка/DEX_new/Dex-Dynamic-Fee/visualizations/../historical_data.py:43: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  time_range = pd.date_range(start=start_time, end=end_time, freq=interval)


Symbol USDTUSDT is USDT/USDT - returning price=1
A symbol candles count: 8269
B symbol candles count: 0
Joined candles count: 0


/Users/ir.v.lebedeva/Documents/Научка/DEX_new/Dex-Dynamic-Fee/visualizations/../historical_data.py:43: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  time_range = pd.date_range(start=start_time, end=end_time, freq=interval)


IndexError: single positional indexer is out-of-bounds

In [11]:
experiment_results_stable_stable

{frozenset({('fee_algorithm', 'discrete_fee_perfect_oracle'),
            ('period_alias', 'calm_market')}): 'Error: -1',
 frozenset({('fee_algorithm',
             'adaptive_based_on_previous_block_price_move_fee'),
            ('period_alias', 'calm_market')}): 'Error: -1',
 frozenset({('fee_algorithm', 'fixed_fee'),
            ('period_alias', 'calm_market')}): 'Error: -1',
 frozenset({('fee_algorithm', 'AMM_fee'),
            ('period_alias', 'calm_market')}): 'Error: -1',
 frozenset({('fee_algorithm', 'based_on_trade_count_fee'),
            ('period_alias', 'calm_market')}): 'Error: -1'}

In [8]:
import pandas as pd
from visualizations.compare_fee_algoritms import get_experiments_summary_by_description

combined_df = get_experiments_summary_by_description(
    experiment_results_stable_stable,
).sort_values(by=["period_alias", "fee_algorithm"])

combined_df = combined_df.set_index(['period_alias'])

display(combined_df)

Error: -1


AttributeError: 'str' object has no attribute 'simulation_result'

In [10]:
from pprint import pprint

pprint(experiment_configs[
    get_experiment_key(
        {
        "period_alias": "calm_market",
        "fee_algorithm": "adaptive_based_on_previous_block_price_move_fee",
    })
])

Experiment(data=HistoricalDataDescription(start_time=Timestamp('2025-05-05 00:00:00'), end_time=Timestamp('2025-05-11 23:59:59.999999999'), A_symbol='TUSD', B_symbol='USDT', stable_coin_symbol='USDT', candle_interval='1m'), fee_algorithm=AdaptiveBasedOnPreviousBlockPriceMoveFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003, fee_step=0.0001, prev_block_begin_a_to_b_price=None, prev_block_end_a_to_b_price=None, prev_block_begin_b_to_a_price=None, prev_block_end_b_to_a_price=None), informed_user=InformedUser(), uninformed_users=UninformedUsersConfig(uninformed_user=UninformedUser(mu=0.00016, sigma=1e-05), probability_of_trade=0.5, n_users=1), initial_pool_value=25000000, network_fee=5, random_seed=0)
